In [1]:
import pandas as pd

request = pd.read_excel('./sample_data.xlsx', sheet_name="request")
impression = pd.read_excel('./sample_data.xlsx', sheet_name="impression")
feedback = pd.read_excel('./sample_data.xlsx', sheet_name="feedback")

feedback.head(10)

,request_id,track_id,clicked,skipped_10s,listened_30s,liked,listened_sec,reward_score
0,REQ_0001,TRK_00051,1,0,1,1,842,6
1,REQ_0001,TRK_00054,1,0,1,0,463,3
2,REQ_0001,TRK_00057,1,0,0,0,184,1
3,REQ_0001,TRK_00060,0,1,0,0,62,-2
4,REQ_0001,TRK_00063,0,1,0,0,41,-2
5,REQ_0002,TRK_00006,1,0,1,1,905,6
6,REQ_0002,TRK_00007,1,0,1,0,512,3
7,REQ_0002,TRK_00008,1,0,0,0,201,1
8,REQ_0002,TRK_00009,0,1,0,0,53,-2
9,REQ_0002,TRK_00010,0,1,0,0,39,-2


In [2]:
# request_id 기준 개수 체크
request["request_id"].nunique()
impression["request_id"].nunique()
feedback["request_id"].nunique()

50

In [3]:
# impression <-> feedback 1:1대응 체크
assert (
    impression[["request_id", "track_id"]]
    .merge(feedback[["request_id", "track_id"]],
           on=["request_id", "track_id"],
           how="outer")
    .isnull()
    .sum()
    .sum() == 0
)

In [4]:
# 학습 단위(track 하나) 테이블 만들기
df = (
    impression
    .merge(feedback, on=["request_id", "track_id"], how="left")
    .merge(request, on="request_id", how="left")
)

df.head(10)

,policy_id,request_id,track_id,artist_id,rank,clicked,skipped_10s,listened_30s,liked,listened_sec,...,user_id,created_at,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag
0,POL_RULE_V1,REQ_0001,TRK_00051,ART_0017,1,1,0,1,1,842,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
1,POL_RULE_V1,REQ_0001,TRK_00054,ART_0018,2,1,0,1,0,463,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
2,POL_RULE_V1,REQ_0001,TRK_00057,ART_0019,3,1,0,0,0,184,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
3,POL_RULE_V1,REQ_0001,TRK_00060,ART_0020,4,0,1,0,0,62,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
4,POL_RULE_V1,REQ_0001,TRK_00063,ART_0021,5,0,1,0,0,41,...,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
5,POL_RULE_V2,REQ_0002,TRK_00006,ART_0002,1,1,0,1,1,905,...,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
6,POL_RULE_V2,REQ_0002,TRK_00007,ART_0003,2,1,0,1,0,512,...,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
7,POL_RULE_V2,REQ_0002,TRK_00008,ART_0003,3,1,0,0,0,201,...,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
8,POL_RULE_V2,REQ_0002,TRK_00009,ART_0003,4,0,1,0,0,53,...,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
9,POL_RULE_V2,REQ_0002,TRK_00010,ART_0004,5,0,1,0,0,39,...,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night


In [5]:
df.query("liked == 1 & (clicked == 0 | listened_30s == 0)")

,policy_id,request_id,track_id,artist_id,rank,clicked,skipped_10s,listened_30s,liked,listened_sec,...,user_id,created_at,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag


In [6]:
df.query("skipped_10s == 1 & listened_30s == 1")

,policy_id,request_id,track_id,artist_id,rank,clicked,skipped_10s,listened_30s,liked,listened_sec,...,user_id,created_at,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag


In [7]:
df.groupby("rank")["clicked"].mean()

rank
1    0.94
2    1.00
3    0.56
4    0.02
5    0.22
Name: clicked, dtype: float64

In [8]:
df["reward_score"].value_counts().sort_index()

reward_score
-2    113
 1     50
 3     49
 6     38
Name: count, dtype: int64

In [9]:
df.groupby("policy_id")["reward_score"].mean()

policy_id
POL_HYBRID_V1    0.700000
POL_RULE_V1      0.858824
POL_RULE_V2      0.823529
Name: reward_score, dtype: float64

In [10]:
import numpy as np
import ast

def mood_entropy(mood_vector):
    x = np.array(mood_vector, dtype=float)
    if x.sum() == 0:
        return 0.0
    p = x / x.sum()
    p = p[p>0]
    return -np.sum(p*np.log(p))

df["mood_vector_parsed"] = df["mood_vector"].apply(ast.literal_eval) 
df["mood_entropy"] = df["mood_vector_parsed"].apply(mood_entropy)

df["mood_entropy"]

0      1.689245
1      1.689245
2      1.689245
3      1.689245
4      1.689245
         ...   
245    1.351784
246    1.351784
247    1.351784
248    1.351784
249    1.351784
Name: mood_entropy, Length: 250, dtype: float64

In [11]:
df.to_excel("./sample_data_2.xlsx")

In [12]:
df.columns

Index(['policy_id', 'request_id', 'track_id', 'artist_id', 'rank', 'clicked',
       'skipped_10s', 'listened_30s', 'liked', 'listened_sec', 'reward_score',
       'user_id', 'created_at', 'context_text', 'mood_vector', 'dominant_mood',
       'mood_level_max', 'mood_entropy', 'situation_tag', 'place_tag',
       'weather_tag', 'mood_vector_parsed'],
      dtype='object')

In [13]:
mood_cols = ["anger","sadness","pain","anxiety","shame","joy","love"]

mood_df = pd.DataFrame(df["mood_vector_parsed"].tolist(),
                       columns=[f"mood_{c}" for c in mood_cols])

mood_df

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love
0,1,7,3,4,1,2,1
1,1,7,3,4,1,2,1
2,1,7,3,4,1,2,1
3,1,7,3,4,1,2,1
4,1,7,3,4,1,2,1
...,...,...,...,...,...,...,...
245,0,2,0,1,0,2,2
246,0,2,0,1,0,2,2
247,0,2,0,1,0,2,2
248,0,2,0,1,0,2,2


In [14]:
dominant_mood = pd.get_dummies(df["dominant_mood"], 
                               columns=["dominant_mood"], 
                               prefix="dominant_mood")

situation_tag = pd.get_dummies(df["situation_tag"],
                               columns=["situation_tag"],
                               prefix="situation")

place_tag = pd.get_dummies(df["place_tag"],
                           columns=["place_tag"],
                           prefix="place")

weather_tag = pd.get_dummies(df["weather_tag"],
                             columns=["weather_tag"],
                             prefix="weather")

weather_tag

,weather_clear,weather_cloudy,weather_night,weather_rain
0,False,True,False,False
1,False,True,False,False
2,False,True,False,False
3,False,True,False,False
4,False,True,False,False
...,...,...,...,...
245,False,False,True,False
246,False,False,True,False
247,False,False,True,False
248,False,False,True,False


In [15]:
X = pd.concat([mood_df, dominant_mood, df["mood_level_max"], df["mood_entropy"],
                  situation_tag, place_tag, weather_tag], axis=1)

X

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love,dominant_mood_anxiety,dominant_mood_joy,dominant_mood_love,...,place_cafe,place_gym,place_home,place_office,place_outdoor,place_transport,weather_clear,weather_cloudy,weather_night,weather_rain
0,1,7,3,4,1,2,1,False,False,False,...,False,False,True,False,False,False,False,True,False,False
1,1,7,3,4,1,2,1,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2,1,7,3,4,1,2,1,False,False,False,...,False,False,True,False,False,False,False,True,False,False
3,1,7,3,4,1,2,1,False,False,False,...,False,False,True,False,False,False,False,True,False,False
4,1,7,3,4,1,2,1,False,False,False,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0,2,0,1,0,2,2,False,True,False,...,False,False,True,False,False,False,False,False,True,False
246,0,2,0,1,0,2,2,False,True,False,...,False,False,True,False,False,False,False,False,True,False
247,0,2,0,1,0,2,2,False,True,False,...,False,False,True,False,False,False,False,False,True,False
248,0,2,0,1,0,2,2,False,True,False,...,False,False,True,False,False,False,False,False,True,False


In [76]:
df["new_reward_score"] = (
    5 * df["liked"] +
    3 * df["listened_30s"] + 
    1 * df["clicked"] +
    0 * df["skipped_10s"]
)

df["new_reward_score"]

0      9
1      4
2      1
3      0
4      0
      ..
245    4
246    1
247    0
248    0
249    9
Name: new_reward_score, Length: 250, dtype: int64

In [77]:
y = df["new_reward_score"].astype(float)

y

0      9.0
1      4.0
2      1.0
3      0.0
4      0.0
      ... 
245    4.0
246    1.0
247    0.0
248    0.0
249    9.0
Name: new_reward_score, Length: 250, dtype: float64

In [78]:
sort_cols = ["request_id", "rank"]

df = df.sort_values(sort_cols).reset_index(drop=True)
X = X.loc[df.index].reset_index(drop=True)
y = y.loc[df.index].reset_index(drop=True)

In [79]:
df

,policy_id,request_id,track_id,artist_id,rank,clicked,skipped_10s,listened_30s,liked,listened_sec,...,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag,mood_vector_parsed,new_reward_score
0,POL_RULE_V1,REQ_0001,TRK_00051,ART_0017,1,1,0,1,1,842,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,"[1, 7, 3, 4, 1, 2, 1]",9
1,POL_RULE_V1,REQ_0001,TRK_00054,ART_0018,2,1,0,1,0,463,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,"[1, 7, 3, 4, 1, 2, 1]",4
2,POL_RULE_V1,REQ_0001,TRK_00057,ART_0019,3,1,0,0,0,184,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,"[1, 7, 3, 4, 1, 2, 1]",1
3,POL_RULE_V1,REQ_0001,TRK_00060,ART_0020,4,0,1,0,0,62,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,"[1, 7, 3, 4, 1, 2, 1]",0
4,POL_RULE_V1,REQ_0001,TRK_00063,ART_0021,5,0,1,0,0,41,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,"[1, 7, 3, 4, 1, 2, 1]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,POL_RULE_V2,REQ_0050,TRK_00266,ART_0009,1,1,0,1,0,469,...,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.351784,sleeping,home,night,"[0, 2, 0, 1, 0, 2, 2]",4
246,POL_RULE_V2,REQ_0050,TRK_00268,ART_0009,2,1,0,0,0,206,...,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.351784,sleeping,home,night,"[0, 2, 0, 1, 0, 2, 2]",1
247,POL_RULE_V2,REQ_0050,TRK_00270,ART_0010,3,0,1,0,0,75,...,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.351784,sleeping,home,night,"[0, 2, 0, 1, 0, 2, 2]",0
248,POL_RULE_V2,REQ_0050,TRK_00272,ART_0011,4,0,1,0,0,54,...,하루 마무리하면서 차분해지고 싶어,"[0,2,0,1,0,2,2]",joy,2,1.351784,sleeping,home,night,"[0, 2, 0, 1, 0, 2, 2]",0


In [80]:
req_ids = df["request_id"].unique()

req_ids

array(['REQ_0001', 'REQ_0002', 'REQ_0003', 'REQ_0004', 'REQ_0005',
       'REQ_0006', 'REQ_0007', 'REQ_0008', 'REQ_0009', 'REQ_0010',
       'REQ_0011', 'REQ_0012', 'REQ_0013', 'REQ_0014', 'REQ_0015',
       'REQ_0016', 'REQ_0017', 'REQ_0018', 'REQ_0019', 'REQ_0020',
       'REQ_0021', 'REQ_0022', 'REQ_0023', 'REQ_0024', 'REQ_0025',
       'REQ_0026', 'REQ_0027', 'REQ_0028', 'REQ_0029', 'REQ_0030',
       'REQ_0031', 'REQ_0032', 'REQ_0033', 'REQ_0034', 'REQ_0035',
       'REQ_0036', 'REQ_0037', 'REQ_0038', 'REQ_0039', 'REQ_0040',
       'REQ_0041', 'REQ_0042', 'REQ_0043', 'REQ_0044', 'REQ_0045',
       'REQ_0046', 'REQ_0047', 'REQ_0048', 'REQ_0049', 'REQ_0050'],
      dtype=object)

In [81]:
rng = np.random.default_rng(42) 
rng.shuffle(req_ids)

In [82]:
n_train = int(len(req_ids)*0.8) 
train_reqs = set(req_ids[:n_train])
test_reqs = set(req_ids[n_train:])

train_mask = df["request_id"].isin(train_reqs)
test_mask = df["request_id"].isin(test_reqs)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

In [65]:
group_train = df.loc[train_mask].groupby("request_id").size().to_list()
group_test = df.loc[test_mask].groupby("request_id").size().to_list()

In [56]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [83]:
df.loc[train_mask].shape

(200, 23)

In [84]:
from xgboost import XGBRanker

model = XGBRanker(
    objective="rank:pairwise",
    eval_metric="ndcg",
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
)

model.fit(
    X_train, y_train,
    group=group_train,
    eval_set=[(X_test, y_test)],
    eval_group=[group_test],
    verbose=False
)

,objective,'rank:pairwise'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg'


In [86]:
model_topk_reward = (
    group_test.sort_values(["request_id", "pred_score"], ascending=[True, False])
           .groupby("request_id")
           .head(K)
           .groupby("request_id")["reward_score"]
           .sum()
           .mean()
)

print("\nModel Top-K reward mean:", model_topk_reward)

# 비교용 베이스라인: "원래 rank 순서대로 Top-K"
if "rank" in group_test.columns:
    baseline_topk_reward = (
        group_test.sort_values(["request_id", "rank"])
               .groupby("request_id")
               .head(K)
               .groupby("request_id")["reward_score"]
               .sum()
               .mean()
    )
    print("Rank baseline Top-K reward mean:", baseline_topk_reward)

AttributeError: 'list' object has no attribute 'sort_values'